In [6]:
import stanza
import pandas as pd
import pyarabic.araby as araby
import pyarabic.number as number
import libqutrub.conjugator
from IPython.core.display import display, HTML
import alyahmor.genelex
import random
from spacy_stanza import StanzaLanguage
from spacy import displacy
import re
import numpy as np
nlp = stanza.Pipeline('ar',processors='tokenize,mwt,pos,lemma,depparse', verbose = True);


2021-07-29 13:29:00 INFO: Loading these models for language: ar (Arabic):
| Processor | Package |
-----------------------
| tokenize  | padt    |
| mwt       | padt    |
| pos       | padt    |
| lemma     | padt    |
| depparse  | padt    |

2021-07-29 13:29:00 INFO: Use device: cpu
2021-07-29 13:29:00 INFO: Loading: tokenize
2021-07-29 13:29:00 INFO: Loading: mwt
2021-07-29 13:29:00 INFO: Loading: pos
2021-07-29 13:29:02 INFO: Loading: lemma
2021-07-29 13:29:02 INFO: Loading: depparse
2021-07-29 13:29:03 INFO: Done loading processors!


In [7]:
# Column:	Status

# ID:	Sentence-level units in PADT often correspond to entire paragraphs and they were obtained automatically. Low-level tokenization (whitespace and punctuation) was done automatically and then hand-corrected. Splitting of fused tokens into syntactic words in Arabic is part of morphological analysis. ElixirFM was used to provide context-independent options, then these results were disambiguated manually.

# FORM:	The unvocalized surface form is used. Fully vocalized counterpart can be found in the MISC column as Vform attribute.

# LEMMA:	Plausible analyses provided by ElixirFM, manual disambiguation. Lemmas are vocalized. Part of the selection of lemmas was also word sense disambiguation of the lexemes, providing English equivalents (see the Gloss attribute of the MISC column).

# UPOSTAG:	Converted automatically from XPOSTAG (via Interset); human checking of patterns revealed by automatic consistency tests.

# XPOSTAG:	Manual selection from possibilities provided by ElixirFM.
# FEATS:	Converted automatically from XPOSTAG (via Interset); human checking of patterns revealed by automatic consistency tests.

# HEAD:	Original PADT annotation is manual. Automatic conversion to UD; human checking of patterns revealed by automatic consistency tests.

# DEPREL:	Original PDT annotation is manual. Automatic conversion to UD; human checking of patterns revealed by automatic consistency tests.

# DEPS:	— (currently unused)

# MISC:	Information about token spacing taken from PADT annotation. Additional word attributes provided by morphological analysis (i.e. ElixirFM rules + manual disambiguation): Vform (fully vocalized Arabic form), Translit (Latin transliteration of word form), LTranslit (Latin transliteration of lemma), Root (word root), Gloss (English translation of lemma).

In [8]:
def dotCountChanger(word):
    """ for each letter in a word,
    change it with a similar character but with different number of dots, then add 
    each of the character replaced and the new character to the lists list_of_chrs_to_change and list_of_chrs_to_add
    respectively"""
    list_of_distractors=[]
    chr_to_change=""
    chr_to_add=""
    for i in word:
        chr_to_change=""
        chr_to_add=""
        if "ت" == i:
            chr_to_add= "ث"
            chr_to_change= "ت"
        if "ن" == i:
            chr_to_add= "ت" 
            chr_to_change= "ن"
        if "س" == i:
            chr_to_add= "ش" 
            chr_to_change="س"
        if "ف" == i:
            chr_to_add= "ق"
            chr_to_change="ف"
        if "ب" == i:
            chr_to_add= "ي"
            chr_to_change="ب"
        if "ح" == i:
            chr_to_add= "خ" 
            chr_to_change="ح"
        if "خ" == i:
            chr_to_add= "ج" 
            chr_to_change="خ" 
        if "ز" == i:
            chr_to_add= "ر" 
            chr_to_change="ز"
        if "ط" == i:
            chr_to_add= "ظ"
            chr_to_change="ط" 
        if "د" == i:
            chr_to_add= "ذ" 
            chr_to_change="د"
        if "ع" == i:
            chr_to_add= "غ"
            chr_to_change="ع"
        if "ص" == i:
            chr_to_add= "ض"
            chr_to_change="ص"

        if "ث" == i:
            chr_to_add= "ت" 
            chr_to_change="ث" 
        if "ش" == i:
            chr_to_add= "س" 
            chr_to_change="ش"
        if "ق" == i:
            chr_to_add= "ف" 
            chr_to_change="ق"
        if "ي" == i:
            chr_to_add= "ب"
            chr_to_change="ي"
        if "خ" == i:
            chr_to_add= "ح" 
            chr_to_change="خ"
        if "ج" == i:
            chr_to_add= "خ" 
            chr_to_change="ج"
        if "ر" == i:
            chr_to_add= "ز"  
            chr_to_change="ر"
        if "ظ" == i:
            chr_to_add= "ط" 
            chr_to_change="ظ"
        if "ذ" == i:
            chr_to_add= "د" 
            chr_to_change="ذ"
        if "غ" == i:
            chr_to_add= "ع"
            chr_to_change="غ"
        if "ض" == i:
            chr_to_add= "ص"
            chr_to_change="ض"
        
        index_of_old=word.index(chr_to_change)
        new_word=''
        if chr_to_change !='' and chr_to_add !='':
            new_word=word.replace(chr_to_change,chr_to_add)
        if new_word not in list_of_distractors and new_word !='':
             list_of_distractors.append(new_word)
    return list_of_distractors
print(dotCountChanger("النمور"))


['التمور', 'النموز']


In [9]:
"""" function to change the shape of the letter ["ا","إ","أ"] if occured in a  given word  """
def AlefShapeChanger(word):
    listOfDistractors=[]
    listAlef = ["ا","إ","أ"]
#     word=araby.strip_tashkeel(word)
    for i in range (len(word)):
        if (word[i]) in listAlef:
            for k in listAlef:
                if (word[:i]+k+word[i+1:]) not in listOfDistractors:
                    listOfDistractors.append(word[:i]+k+word[i+1:])
    if word in listOfDistractors:
        listOfDistractors.remove(word)
    return listOfDistractors
AlefShapeChanger("النمار")

['إلنمار', 'ألنمار', 'النمإر', 'النمأر']

In [10]:
"""" function to change the last letter in the word if it is  ه with  ة  and vice versa"""
def lastLetterChanger(word):
    list_of_letterswDots=["ت","ن","س","ف","ب","ح","خ","ز","ط","د","ع","ث","ش","ق","ي","ج","ر","ظ","ذ" ,"غ" ]
    word=araby.strip_tashkeel(word)
    list_of_distractors=[]
    temp=""
    for i in range (2,len(word)):
        if "ه" == word[-1] :
            word=word[:-1]+"ة"
            break
        if "ة" == word[-1] :
            word=word[:-1]+"ه"
            break
    return word
lastLetterChanger("النمارة")

'النماره'

# 6000 sentences data cleaning

In [11]:
df= pd.read_excel("7000 sentences Corpus with IDs (1).xlsx",usecols=['ID','Arabic'])
sep = '='
temp=[]
df=df.dropna()
for i in df['Arabic']:
    temp.append(i.split(sep, 1)[0])
df['Arabic']=temp
cleaned_sentences=[]
for i in df['Arabic']:
    x=i
    for k in x:
        if k in ['(',')']:
            openBracket=x.index('(')
            closeBracket = x.index(')')
            x=x[:openBracket]+x[(closeBracket+1):]
            break
    cleaned_sentences.append(x.replace("  ", " "))
df['Arabic_cleaned']=(cleaned_sentences)

# Exercises creation and dataset output for verb conjugation exercises

In [19]:
# the list to be exported as dataframe
finalDataFrameList=[]
cpt = 0
for ind in df.index:
    
    print(cpt)
    # sentence with diacritics 
    Full_sentence=df['Arabic_cleaned'][ind]########Full_sentence
    tokenizedSentence= araby.tokenize(Full_sentence)
    # sentence after removing diacritics
    sentence_stripped=araby.strip_tashkeel(Full_sentence)
    #POS tagging the sentence without diacritics
    dictt=nlp(sentence_stripped).to_dict()[0]
    # converting the sentence without diacritics to a list
    tokenized_sentence_stripped=araby.tokenize(sentence_stripped)
    if len(tokenizedSentence)>2:


        if tokenizedSentence[-1] in ['.','؟','!'] and tokenized_sentence_stripped[-1] in ['.','؟','!']:

            # getting the base of the verb in the sentence
            list_of_verbs=[]
            list_of_baseVerbs=[]
            list_of_verbIndex=[]
            for i in dictt:
                if 'upos' in i:
                    if i['upos']=='VERB':
                        if i['text'] in tokenized_sentence_stripped:
                            # getting the index of the verb in the list of the sentence without diacritics
                            list_of_verbIndex.append(tokenized_sentence_stripped.index(i['text']))
                            list_of_verbs.append(i['text'])
                            list_of_baseVerbs.append(i['lemma'])


            # a loop over the sentence, if the sentence has more than one verb, more than one exercise with be created
            for verb in range(len(list_of_verbs)):
                if len(araby.tokenize(Full_sentence)) >list_of_verbIndex[verb]:
                    # saving the correct answer in Right_answer after getting its index
                    Right_answer=araby.tokenize(Full_sentence)[list_of_verbIndex[verb]]########Right_answer
                    
                    # using the library libqutrub to conjugate a verb
                    future_type =u""
                    all = False # all tenses
                    past = True
                    future=True
                    passive =False
                    imperative=False
                    future_moode= False
                    confirmed=False
                    transitive =False
                    """"all = True # conjugate in all arabic tenses.
                        past = True # conjugate in past tense ألماضي
                        future=True # conjugate in arabic present and future tenses المضارع
                        passive =True # conjugate in passive voice المبني للمجهول
                        imperative=True # conjugate in imperative tense الأمر
                        future_moode= True # conjugate in future moode tenses المضارع المنصوب والمجزوم
                        confirmed=True # conjugate in confirmed cases tense المؤكّد
                        transitive =True # the verb transitivity """
                    
                    
                    
                    
                    display_format="DICT" #output the conjugation table as DICT
                    ''' display format forms:
                        'Text':
                        'HTML':
                        'HTMLColoredDiacritics':
                        'DICT':
                        'CSV':
                        'GUI':
                        'TABLE':
                        'XML':
                        'TeX':
                        'ROWS': '''
                    # calling the function .conjugator.conjugate() to conjugate the verb
                    table = libqutrub.conjugator.conjugate(araby.strip_tashkeel(list_of_baseVerbs[verb]),future_type,
                    all,past,future,passive,imperative,future_moode,confirmed,transitive,display_format);
                        
                    if table is None:
                        Right_answer=araby.tokenize(Full_sentence)[list_of_verbIndex[verb]]########Right_answer
                        future_type =u""
                        all = False # all tenses
                        past = True
                        future=True
                        passive =False
                        imperative=False
                        future_moode= False
                        confirmed=False
                        transitive =False
                        display_format="DICT"
                        table = libqutrub.conjugator.conjugate((list_of_baseVerbs[verb]),future_type,
                        all,past,future,passive,imperative,future_moode,confirmed,transitive,display_format);


                    #loop to create the list of the displayed question with base verb and "..."
                    tokenizedSentenceTemp=[]
                    if tokenizedSentence.index(Right_answer)!=0 and tokenizedSentence.index(Right_answer)!=len(tokenizedSentence)-1:
                        for i in tokenizedSentence:
                            if araby.strip_tashkeel(i) == araby.strip_tashkeel(Right_answer):
                                tokenizedSentenceTemp.append("..."+'('+list_of_baseVerbs[verb]+')')
                            else:
                                tokenizedSentenceTemp.append(i)



                        #list of all the verbs conjugated in present simpla and past simple with all pronouns 
                        # with removing the ones identical in letters to the correct answer
                        if table is not None:
                            listOfAllConjugations = list(table['الماضي المعلوم'].values())+list(table['المضارع المعلوم'].values())
                            listOfDistractors=[]
                            for i in listOfAllConjugations:
                                # checking if the distractor to be added isn't identical to the correct answer in letters
                                  if araby.strip_tashkeel(i) != araby.strip_tashkeel(Right_answer) and i not in listOfDistractors :
                                        listOfDistractors.append(i)


                            # picking 3 distractors at random from the list of all the distractors
                            listOfChoices = random.sample(listOfDistractors, 3)
                            dist_1=listOfChoices[0]########dist_1
                            dist_2=listOfChoices[1]########dist_2
                            dist_3=listOfChoices[2]########dist_3
                            listOfChoices.append(Right_answer)
                            #shuffling the list of the 4 choices
                            listOfChoices = random.sample(listOfChoices,4)

                            cpt+=1

                            Sentence_with_blank=(' '.join(map(str,tokenizedSentenceTemp)))########Sentence_with_blank

                            Exo_type_id = 0

                            Exo_type = "Cloze_Test_+_MCQ"

                            Exo_objective= "Grammar"

                            Exo_focus = "Verb_conjugation"

                            Exo_id = cpt

                            Source_format= 'Text'

                            Target_format= 'Text'

                            Source_sentence_id = df['ID'][ind]

                            Source_word_id = 0

                            Source_lang= 'French'

                            Target_lang= 'Arabic'

                            Instruction = "Conjugate the ver between brackets in the correct tense and with the suitable pronoun"

                            Explanation = ""

                            Difficulty = 0

                            Remediation = ""

                            propositions = '-'.join(listOfChoices)



                            finalList=[Exo_type_id, Exo_type, Exo_objective,Exo_focus,Exo_id,Source_format,Target_format,
                                       Source_sentence_id ,Source_word_id, Source_lang,Target_lang,Full_sentence,Instruction,Sentence_with_blank,
                                       Right_answer,dist_1,dist_2,dist_3, propositions,Explanation, Difficulty, Remediation]

                            finalDataFrameList.append(finalList)
finalDataFrame=pd.DataFrame(finalDataFrameList,columns=["Exo_type_id","Exo_type",'Exo_objective',"Exo_focus",'Exo_id','Source_format','Target_format',
                                                             "Source_sentence_id" ,"Source_word_id", 'Source_lang','Target_lang','Full_sentence',"Instruction", 'Sentence_with_blank',
                                                             'Right_answer', 'dist_1', 'dist_2', 'dist_3', "Propositions","Explanation", "Difficulty", "Remediation"])
finalDataFrame.to_excel('finalDataFrameVerbConjExos6k.xlsx')


0
0
0
0
0
0
1
2
2
2
2
3
4
4
4
5
5
5
5
5
5
5
5
5
5
5
6
6
6
6
6
6
7
8
8
9
9
9
9
10
10
10
12
12
12
12
12
13
14
14


## Verb conjugation creation with focus point on gender

In [17]:
## for the dataset
finalDataFrameList=[]
cpt = 0
for ind in df.index:
    # sentence with diacritics 
    Full_sentence=df['Arabic_cleaned'][ind]########Full_sentence
    tokenizedSentence= araby.tokenize(Full_sentence)
    # sentence after removing diacritics
    sentence_stripped=araby.strip_tashkeel(Full_sentence)
    #POS tagging the sentence without diacritics
    dictt=nlp(sentence_stripped).to_dict()[0]
    # converting the sentence without diacritics to a list
    tokenized_sentence_stripped=araby.tokenize(sentence_stripped)
    
    
    # getting the base of the verb in the sentence
    list_of_verbs=[]
    list_of_baseVerbs=[]
    list_of_verbIndex=[]
    list_of_feats=[]
    for i in dictt:
        if 'upos' in i:
            if i['upos']=='VERB':
                if i['text'] in tokenized_sentence_stripped:
                    # getting the index of the verb in the list of the sentence without diacritics
                    list_of_verbIndex.append(tokenized_sentence_stripped.index(i['text']))
                    list_of_verbs.append(i['text'])
                    list_of_baseVerbs.append(i['lemma'])  
                 
                    
    for verb in range(len(list_of_verbs)):
    

        if len(araby.tokenize(Full_sentence)) >list_of_verbIndex[verb]:
            if tokenizedSentence[-1] in ['.','؟','!'] and tokenized_sentence_stripped[-1] in ['.','؟','!']:

                # saving the correct answer in Right_answer after getting its index
                Right_answer=araby.tokenize(Full_sentence)[list_of_verbIndex[verb]]########Right_answer
                future_type =u""
                all = False # all tenses
                past = True
                future=True
                passive =False
                imperative=False
                future_moode= False
                confirmed=False
                transitive =False
                display_format="DICT"
                table = libqutrub.conjugator.conjugate(araby.strip_tashkeel(list_of_baseVerbs[verb]),future_type,
                all,past,future,passive,imperative,future_moode,confirmed,transitive,display_format);

                #loop to create the list of the displayed question with base verb and "..."
                tokenizedSentenceTemp=[]
                if tokenizedSentence.index(Right_answer)!=0 and tokenizedSentence.index(Right_answer)!=len(tokenizedSentence)-1:
                    for i in tokenizedSentence:
                        if araby.strip_tashkeel(i) == araby.strip_tashkeel(Right_answer):
                            tokenizedSentenceTemp.append("..."+'('+list_of_baseVerbs[verb]+')')
                        else:
                            tokenizedSentenceTemp.append(i)

                    #list of all the verbs conjugated in present simpla and past simple with all pronouns 
                    # with removing the ones identical in letters to the correct answer
                    if table is not None:
                        listOfMasc=[(table['الماضي المعلوم']['هو']),(table['المضارع المعلوم']['هم'])]

                        listOfFem=[(table['المضارع المعلوم']['هي']),(table['المضارع المعلوم']['هن'])]

                        listOfDistractors=[]
                        # checking if the distractor to be added isn't identical to the correct answer in letters
                        for i in listOfMasc+listOfFem:

                            if araby.strip_tashkeel(Right_answer) != araby.strip_tashkeel(i):
                                listOfDistractors.append(i)

                        # picking 3 distractors at random from the list of all the distractors
                        if len(listOfDistractors)>2:
                            listOfChoices = random.sample(listOfDistractors, 3)
                            dist_1=listOfChoices[0]########dist_1
                            dist_2=listOfChoices[1]########dist_2
                            dist_3=listOfChoices[2]########dist_3
                            listOfChoices.append(Right_answer)
                            #shuffling the list of the 4 choices
                            listOfChoices = random.sample(listOfChoices,4)

                            cpt+=1

                            Sentence_with_blank=(' '.join(map(str,tokenizedSentenceTemp)))########Sentence_with_blank

                            Exo_type_id = 35

                            Exo_type = "Cloze_Test_+_MCQ"

                            Exo_objective= "Verb_Conjugation"

                            Exo_focus = "Gender"

                            Exo_id = cpt

                            Source_format= 'Text'

                            Target_format= 'Text'

                            Source_sentence_id = df['ID'][ind]

                            Source_word_id = 0

                            Source_lang= 'French'

                            Target_lang= 'Arabic'

                            Instruction = "Conjugate the verb between brackets:"

                            Explanation = ""

                            Difficulty = 0

                            Remediation = ""

                            propositions = '-'.join(listOfChoices)


                            finalList=[Exo_type_id, Exo_type, Exo_objective,Exo_focus,Exo_id,Source_format,Target_format,
                                       Source_sentence_id ,Source_word_id, Source_lang,Target_lang,Full_sentence,Instruction,Sentence_with_blank,
                                       Right_answer,dist_1,dist_2,dist_3,propositions, Explanation, Difficulty, Remediation]

                            finalDataFrameList.append(finalList)

                            print(cpt)
finalDataFrame=pd.DataFrame(finalDataFrameList,columns=["Exo_type_id","Exo_type",'Exo_objective',"Exo_focus",'Exo_id','Source_format','Target_format',
                                                             "Source_sentence_id" ,"Source_word_id", 'Source_lang','Target_lang','Full_sentence',"Instruction", 'Sentence_with_blank',
                                                             'Right_answer', 'dist_1', 'dist_2', 'dist_3', "Propositions","Explanation", "Difficulty", "Remediation"])
finalDataFrame.to_excel('finalDataFrameVerbConjExosGenderFocus.xlsx')
        

1
2
3
4
5
6
7
8
9
10
11
12


#  Exercises creation and dataset output for nouns


In [20]:
finalDataFrameList=[]
cpt = 0
for ind in df.index:
    # sentence with diacritics 
    Full_sentence=df['Arabic_cleaned'][ind]########Full_sentence
    tokenizedSentence= araby.tokenize(Full_sentence)

    # sentence after removing diacritics
    sentence_stripped=araby.strip_tashkeel(Full_sentence)

    #POS tagging the sentence without diacritics
    dictt=nlp(sentence_stripped).to_dict()[0]

    # converting the sentence without diacritics to a list
    tokenized_sentence_stripped=araby.tokenize(sentence_stripped)
    
    

    if len(tokenizedSentence)>2:
        
        if tokenizedSentence[-1]  in ['.','؟','!'] and tokenized_sentence_stripped[-1]  in ['.','؟','!']:
            list_of_nounIndex=[]
            list_of_nouns=[]
            # getting the base of the verb in the sentence
            for i in dictt:
                if 'upos' in i:
                    if i['upos']=='NOUN' and len(i['text'])>2:
                        if i['text'] in tokenized_sentence_stripped:
                            list_of_nounIndex.append(tokenized_sentence_stripped.index(i['text']))
                            list_of_nouns.append(i['text'])
                            
            for noun in range (len(list_of_nounIndex)):                
                # saving the correct answer in Right_answer after getting its index
                if list_of_nounIndex[noun] !=0 and list_of_nounIndex[noun] != len(tokenizedSentence)-2:
                    Right_answer=araby.tokenize(Full_sentence)[list_of_nounIndex[noun] ]########Right_answer

                    if Right_answer not in ['.','',' '] and len(Right_answer)>3 :


                        #loop to create the list of the displayed question with base verb and "..."
                        tokenizedSentenceTemp=[]
                        for i in tokenizedSentence:
                            if araby.strip_tashkeel(i) == araby.strip_tashkeel(Right_answer):
                                tokenizedSentenceTemp.append("...")
                            else:
                                tokenizedSentenceTemp.append(i)



                        #list of all the verbs conjugated in present simpla and past simple with all pronouns 
                        # with removing the ones identical in letters to the correct answer
                        listOfAllDistractors=[]
                        listOfAllDistractors+=dotCountChanger(Right_answer)
                        if len(listOfAllDistractors)<4:
                            listOfAllDistractors.append(lastLetterChanger(Right_answer))
                            if len(listOfAllDistractors)<4:
                                listOfAllDistractors+=AlefShapeChanger(Right_answer)


                        if len(listOfAllDistractors)>3:

                            listOfDistractors=[]
                            for i in listOfAllDistractors:
                                # checking if the distractor to be added isn't identical to the correct answer in letters
                                  if araby.strip_tashkeel(i) != araby.strip_tashkeel(Right_answer):
                                      listOfDistractors.append(i)
                            if len(listOfDistractors)>2:
                                # picking 3 distractors at random from the list of all the distractors
                                listOfChoices = random.sample(listOfDistractors, 3)
                                dist_1=listOfChoices[0]########dist_1
                                dist_2=listOfChoices[1]########dist_2
                                dist_3=listOfChoices[2]########dist_3
                                listOfChoices.append(Right_answer)
                                listOfChoices = random.sample(listOfChoices, 4)
                                Sentence_with_blank=(' '.join(map(str,tokenizedSentenceTemp)))########Sentence_with_blank

                                Exo_type_id = 0

                                Exo_type = "Cloze_Test + MCQ"

                                Exo_objective= "Vocabulary"

                                Exo_focus = "Noun_spelling"

                                Exo_id = cpt

                                Source_format= 'Text'

                                Target_format= 'Text'

                                Source_sentence_id = df['ID'][ind]

                                Source_word_id = 0

                                Source_lang= 'French'

                                Target_lang= 'Arabic'

                                Instruction = "Choose the correct answer:"

                                Explanation = ""

                                Difficulty = 0

                                Remediation = ""
                                
                                propositions = '-'.join(listOfChoices)


                                finalList=[Exo_type_id, Exo_type, Exo_objective,Exo_focus,Exo_id,Source_format,Target_format,
                                           Source_sentence_id ,Source_word_id, Source_lang,Target_lang,Full_sentence,Instruction,Sentence_with_blank,
                                           Right_answer,dist_1,dist_2,dist_3, propositions,Explanation, Difficulty, Remediation]
                                finalDataFrameList.append(finalList)
                                cpt+=1
                                print(cpt)
finalDataFrame=pd.DataFrame(finalDataFrameList,columns=["Exo_type_id","Exo_type",'Exo_objective',"Exo_focus",'Exo_id','Source_format','Target_format',
                                                 "Source_sentence_id" ,"Source_word_id", 'Source_lang','Target_lang','Full_sentence',"Instruction", 'Sentence_with_blank',
                                                 'Right_answer', 'dist_1', 'dist_2', 'dist_3', "Propositions","Explanation", "Difficulty", "Remediation"])

finalDataFrame.to_excel('finalDataFrameNounExos.xlsx')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36


#  Exercises creation and dataset output for prepositions

In [22]:
finalDataFrameList=[]
listOfPrepositions=[]
features=''
cpt = 0
for ind in df.index:
    cpt+=1
    # sentence with diacritics 
    Full_sentence=df['Arabic_cleaned'][ind]########Full_sentence
    tokenizedSentence= araby.tokenize(Full_sentence)
    # sentence after removing diacritics
    sentence_stripped=araby.strip_tashkeel(Full_sentence)
    #POS tagging the sentence without diacritics
    dictt=nlp(sentence_stripped).to_dict()[0]
    # converting the sentence without diacritics to a list
    tokenized_sentence_stripped=araby.tokenize(sentence_stripped)
    print(cpt)
    
    # getting the base of the verb in the sentence
    for i in dictt:
        if 'upos' in i:
            if i['upos']=='ADP':
                if i['text'] in tokenized_sentence_stripped:
                    mainPrepositionIndex=tokenized_sentence_stripped.index(i['text']) 
                    features=str(i['feats']).split('|')
                    preposition=i['lemma']
    for i in features:
        if i[:7] == 'AdpType':
            if (i.split('=')[1]) == "Prep":
                listOfPrepositions.append(preposition)
listOfPrepositionsUniquevals= []
for i in listOfPrepositions:
    if i not in listOfPrepositionsUniquevals:
        listOfPrepositionsUniquevals.append(i)
listOfPreps=listOfPrepositionsUniquevals         


# listOfPreps=['فِي','إِزَاءَ','بَينَ', 'مِن','إِلَى', 'لَدَى','أَثنَاءَ','دُونَ','رَغمَ','عَلَى','لِ','خِلَالَ','عَن','دَاخِلَ','حِينَ','قَبلَ','بَعدَ','خَلفَ'
#              ,'فَوقَ','مُنذُ','خَارِجَ','عِندَ','نَحوَ','مِثلَ','مَعَ','طِوَالَ','قُربَ','طِيلَةَ','تَحتَ','ضِمنَ','أَمَامَ','سِوَى','لَعل','حَولَ'
#              ,'جَرَّاء','لِكَّ','وِفقَ','ضِدَّ','تُجَاهَ','حَوَالَى','حَسَبَ','فَئَى','إِثرَ','عَبرَ','تَلوَ','قُبَالَةَ','وَرَاءَ','جَنب']
## for the dataset
finalDataFrameList=[]
cpt = 0
for ind in df.index:
    # sentence with diacritics 
    Full_sentence=df['Arabic_cleaned'][ind]########Full_sentence
    tokenizedSentence= araby.tokenize(Full_sentence)
    # sentence after removing diacritics
    sentence_stripped=araby.strip_tashkeel(Full_sentence)
    #POS tagging the sentence without diacritics
    dictt=nlp(sentence_stripped).to_dict()[0]
    # converting the sentence without diacritics to a list
    tokenized_sentence_stripped=araby.tokenize(sentence_stripped)
    if tokenizedSentence[-1]  in ['.','؟','!'] and tokenized_sentence_stripped[-1]  in ['.','؟','!']:
#         preposition=''
#         Right_answer = ""
#         # getting the base of the verb in the sentence
#         for i in dictt:
#             if 'upos' in i:
#                 if i['upos']=='ADP':
#                     if i['text'] in tokenized_sentence_stripped:
#                         mainPrepositionIndex=tokenized_sentence_stripped.index(i['text']) 
#                         features=str(i['feats']).split('|')
#                         Right_answer = i['lemma']
                        
                        
        list_of_rightAnswers=[]
        list_of_prepIndex=[]
        list_of_feats=[]
        for i in dictt:
            if 'upos' in i:
                if i['upos']=='ADP':
                    if i['text'] in tokenized_sentence_stripped:
                        list_of_prepIndex.append(tokenized_sentence_stripped.index(i['text'])) 
                        list_of_rightAnswers.append(i['lemma'])
                        list_of_feats.append(str(i['feats']).split('|'))
        
        for prep in range(len(list_of_prepIndex)):
            if list_of_prepIndex[prep]!=0 and list_of_prepIndex[prep]!= len(tokenized_sentence_stripped)-1:

                if list_of_rightAnswers[prep] != "":
                    for i in list_of_feats[prep]:
                        if i[:7] == 'AdpType':
                            if (i.split('=')[1]) == "Prep":

            #                     #loop to create the list of the displayed question with base verb and "..."
                                tokenizedSentenceTemp=[]
                                for i in tokenizedSentence:
                                    if araby.strip_tashkeel(i) == araby.strip_tashkeel(list_of_rightAnswers[prep]) and ("...") not in tokenizedSentenceTemp:
                                        tokenizedSentenceTemp.append("...")

                                    else:
                                        tokenizedSentenceTemp.append(i)

                                listOfDistractors=[]
                                # checking if the distractor to be added isn't identical to the correct answer in letters
                                for i in listOfPreps:
                                    if araby.strip_tashkeel(list_of_rightAnswers[prep]) != araby.strip_tashkeel(i):
                                        listOfDistractors.append(i)

                                # picking 3 distractors at random from the list of all the distractors
                                if len(listOfDistractors)>2:
                                    listOfChoices = random.sample(listOfDistractors, 3)
                                    dist_1=listOfChoices[0]########dist_1
                                    dist_2=listOfChoices[1]########dist_2
                                    dist_3=listOfChoices[2]########dist_3
                                    listOfChoices.append(list_of_rightAnswers[prep])
                                    #shuffling the list of the 4 choices
                                    listOfChoices = random.sample(listOfChoices,4)

                                    cpt+=1

                                    Sentence_with_blank=(' '.join(map(str,tokenizedSentenceTemp)))########Sentence_with_blank

                                    Exo_type_id = 0

                                    Exo_type = "Cloze_Test + MCQ"

                                    Exo_objective= "Grammar"

                                    Exo_focus = "Prepositions"

                                    Exo_id = cpt

                                    Source_format= 'Text'

                                    Target_format= 'Text'

                                    Source_sentence_id = df['ID'][ind]

                                    Source_word_id = 0

                                    Source_lang= 'French'

                                    Target_lang= 'Arabic'

                                    Instruction = ""

                                    Explanation = ""

                                    Difficulty = 0

                                    Remediation = ""
                                    
                                    propositions = '-'.join(listOfChoices)




                                    finalList=[Exo_type_id, Exo_type, Exo_objective,Exo_focus,Exo_id,Source_format,Target_format,
                                               Source_sentence_id ,Source_word_id, Source_lang,Target_lang,Full_sentence,Instruction,Sentence_with_blank,
                                               list_of_rightAnswers[prep],dist_1,dist_2,dist_3,propositions ,Explanation, Difficulty, Remediation]

                                    finalDataFrameList.append(finalList)

                                    print(cpt)
finalDataFrame=pd.DataFrame(finalDataFrameList,columns=["Exo_type_id","Exo_type",'Exo_objective',"Exo_focus",'Exo_id','Source_format','Target_format',
                                                             "Source_sentence_id" ,"Source_word_id", 'Source_lang','Target_lang','Full_sentence',"Instruction", 'Sentence_with_blank',
                                                             'Right_answer', 'dist_1', 'dist_2', 'dist_3', 'Propositions',"Explanation", "Difficulty", "Remediation"])
finalDataFrame.to_excel('finalDataFramePrepositions.xlsx')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
3
4
5
6
7
8
9
10
11


#  Exercises creation and dataset output for punctutation

In [ ]:
finalDataFrameList=[]
cpt = 0
for ind in df.index:
    # sentence with diacritics 
    Full_sentence=df['Arabic_cleaned'][ind]########Full_sentence
    tokenizedSentence= araby.tokenize(Full_sentence)
    # sentence after removing diacritics
    sentence_stripped=araby.strip_tashkeel(Full_sentence)
    #POS tagging the sentence without diacritics
    dictt=nlp(sentence_stripped).to_dict()[0]
    # converting the sentence without diacritics to a list
    tokenized_sentence_stripped=araby.tokenize(sentence_stripped)
#     ['.','؟','!','،','؛']
    punctuationMark=''
    mainPunctIndex = 0
    # getting the base of the verb in the sentence
    for i in dictt:
        if 'upos' in i:
            if i['upos']=='PUNCT':
                if i['text'] in tokenized_sentence_stripped:
                    mainPunctIndex=tokenized_sentence_stripped.index(i['text']) 
                    punctuationMark = i['text']
                    break
                break
            
    if tokenizedSentence[0] not in ['.','؟','!','،','؛','"']:
        if punctuationMark !='':
    
            #loop to create the list of the displayed question with  "..."
            tokenizedSentenceTemp=[]
            for i in tokenizedSentence:
                if i == punctuationMark and ("...") not in tokenizedSentenceTemp:
                    tokenizedSentenceTemp.append("...")

                else:
                    tokenizedSentenceTemp.append(i)

            listOfDistractors=[]
            # checking if the distractor to be added isn't identical to the correct answer in letters
            if punctuationMark in ['.','؟','!']:
                for i in ['.','؟','!','،'] :
                    if punctuationMark != i:
                        listOfDistractors.append(i)
            else: 
                for i in ['.','،','؛','؟']:
                    if punctuationMark != i:
                        listOfDistractors.append(i)

            # picking 3 distractors at random from the list of all the distractors
            if len(listOfDistractors)>2:
                listOfChoices = random.sample(listOfDistractors, 3)
                dist_1=listOfChoices[0]########dist_1
                dist_2=listOfChoices[1]########dist_2
                dist_3=listOfChoices[2]########dist_3
                listOfChoices.append(punctuationMark)
                #shuffling the list of the 4 choices
                listOfChoices = random.sample(listOfChoices,4)

                cpt+=1

                Sentence_with_blank=(' '.join(map(str,tokenizedSentenceTemp)))########Sentence_with_blank

                Exo_type_id = 0

                Exo_type = "Cloze_Test + MCQ"

                Exo_objective= "Grammar"

                Exo_focus = "Punctuation"

                Exo_id = cpt

                Source_format= 'text'

                Target_format= 'text'

                Source_sentence_id = df['ID'][ind]

                Source_word_id = 0

                Source_lang= 'French'

                Target_lang= 'Arabic'

                Instruction = "Choose the correct punctuation mark"

                Explanation = ""

                Difficulty = 0

                Remediation = ""
    
                propositions = '-'.join(listOfChoices)




                finalList=[Exo_type_id, Exo_type, Exo_objective,Exo_focus,Exo_id,Source_format,Target_format,
                           Source_sentence_id ,Source_word_id, Source_lang,Target_lang,Full_sentence,Instruction,Sentence_with_blank,
                           punctuationMark,dist_1,dist_2,dist_3, propositions ,Explanation, Difficulty, Remediation]

                finalDataFrameList.append(finalList)

                print(cpt)
finalDataFrame=pd.DataFrame(finalDataFrameList,columns=["Exo_type_id","Exo_type",'Exo_objective',"Exo_focus",'Exo_id','Source_format','Target_format',
                                                             "Source_sentence_id" ,"Source_word_id", 'Source_lang','Target_lang','Full_sentence',"Instruction", 'Sentence_with_blank',
                                                             'Right_answer', 'dist_1', 'dist_2', 'dist_3', "Propositions" , "Explanation", "Difficulty", "Remediation"])
finalDataFrame.to_excel('finalDataFramePunctuation.xlsx')